In [2]:

import requests 
import pandas as pd 

from urllib.request import urlopen
from urllib.parse import quote

def CIRconvert(ids):
    try:
        url = 'http://cactus.nci.nih.gov/chemical/structure/' + quote(ids) + '/smiles'
        ans = urlopen(url).read().decode('utf8')
        return ans
    except:
        return 'Did not work'


import os.path 

datafile = "data/smiles.csv"

if os.path.exists(datafile): 
    print(datafile, " exists") 
else: 
    print("Generating ",datafile)

    QUERY_URL = 'http://sabiork.h-its.org/sabioRestWebServices/searchReactionParticipants'
 
    query = {"SabioReactionID":"*", "fields[]":["Name","Role", "Smiles" ]}

    request = requests.post(QUERY_URL, params = query) 
    request.raise_for_status() 

    print(request.text)

    # results

    fh=open(datafile, "w")

    fh.write(request.text)

data/smiles.csv  exists


In [ ]:
smilesfile = "data/smiles.csv"
#smilesfile = "data/smilestest.csv"

In [ ]:
datafile = "data/ECnumber.csv"

if os.path.exists(datafile):
    print(datafile, "exists")
else:
    print("Generting", datafile)

    QUERY_URL = 'http://sabiork.h-its.org/sabioRestWebServices/searchReactionDetails' 

    query = {"SabioReactionID":"*", "fields[]":["SabioReactionID", "ECNumber"]} 

    # make GET request 
    request = requests.get(QUERY_URL, params = query) 
    request.raise_for_status() 

    #results 
    print(request.text) 

    fh=open("data/ECnumber.csv", "w")

    fh.write(request.text)

    df = pd.read_csv("data/ECnumber.csv", sep='\t')
    df1 = df.sort_values('SabioReactionID')
    print(df1)

In [ ]:
input1=smilesfile
input2='data/ECNumber.csv'

df1 = pd.read_csv(input1, sep='\t')
df2 = pd.read_csv(input2, sep="\t")

# Merge the two dataframes on the common column "numbers"
merged_df = pd.merge(df1, df2, on="SabioReactionID")

# Write the merged dataframe to a new CSV file
merged_df.to_csv("data/SmilesEC.csv", index=False)

## Generem un DF curt per a fer proves

Crear smiles reacció:

Generem un fitxer només per fer proves i agilitzar el procés. En aquest fitxer només es tenen en compte les 10 primeres reaccions presents a SABIO-RK. El document generat s'anomena: smilestest.csv a partir del qual es genera el document de reaccions: ReactionSmilesTest.csv. 

In [ ]:

input= "data/smilestest.csv"
output='data/ReactionSmilesTest.csv'

df1 = pd.read_csv(input, sep='\t')

#Volem eliminar tota la informació de les reaccions que tinguin algun valor nul en els seus subtrats i productes a la columna smiles.

#Primer afegim 0 on tronem valors NaN a la columna Smiles del dataframe.
df1['Smiles'] = df1['Smiles'].fillna(0)


# generem una llista buida on guardarem el SabioReactionID dels substrats i productes que s'hagin d'elimianr 
list1 = [] 

# iterem el df1   
for i in range(len(df1)): 

    # Si Smiles es igual a 0 afegim el SabioReactionID a la list1 
    if df1.loc[i,'Smiles'] == 0: 
        list1.append(df1.loc[i,'SabioReactionID']) 

#Eliminem del df1 tots els substrats i productes que tinguin el SabioReactionID igual que el que s'ha guardat a la llista
df = df1.drop(df1[df1['SabioReactionID'].isin(list1)].index) 


#A partir del dataframe sense missing values generem el dataframe curt per fer proves.
#Generem un dataframe buit amb les columnes SabioID i Reaction de 11 files.
df = df.reset_index()  #la funció rest_index() restableix l'índex del dataframe, estableix una llista del 0 fins a la longitud del dataframe
numrows=11
df2 = pd.DataFrame(columns=["SabioReactionID", "Reaction"]) #, index=range(numrows))
#print(df2)

#Creem dues llistes buides amb tants strings com files (en aquest cas 11 numrows=11). Generem una llista R per guardar els reactius i una llista P per guardar els productes.
R=[""]*(numrows+1)
P=[""]*(numrows+1)
isFirstR=[True]*(numrows+1)
isFirstP=[True]*(numrows+1)

def check_smiles(smiles):
    print(smiles)
    if not smiles:
        print('Wrong SMILES for:',smiles)
        return 0
    return 1

#Fem una iteració per cada fila del dataframe amb el command: for index, row in df.iterrows():
for index, row in df.iterrows():
    rid=row['SabioReactionID']
    name=row["Name"]
    smiles=row["Smiles"]
    if check_smiles(smiles) == 0:
        break
   
    role=row["Role"]
    if role == "Substrate": 
        if isFirstR[rid]:
            R[rid]=R[rid] + smiles 
            isFirstR[rid]= False
        else:
            R[rid]=R[rid] + "." + smiles       
        
    elif role == "Product":
        if isFirstP[rid]:
            P[rid]=P[rid]+smiles
            isFirstP[rid]=False
        else:
            P[rid]=P[rid] +"."+ smiles
    else:
        print("error, name")
    print(R[rid],P[rid])
    df2.at[rid, "SabioReactionID"]=rid

for i in range(numrows):
    print(R[i]+">"+P[i])
    df2.at[i, "Reaction"]=R[i]+">>"+P[i]


df2.drop(df2.tail(2).index,inplace=True)
print(df2)

#obtenim ReactionSmilesTest.csv.
df2.to_csv(output)

In [ ]:
input1="data/ReactionSmilesTest.csv"
input2='data/ECnumber.csv'

df1 = pd.read_csv(input1, sep=',')
df2 = pd.read_csv(input2, sep="\t")

# Merge the two dataframes on the common column "numbers"
merged_df = pd.merge(df1, df2, on="SabioReactionID")

# Write the merged dataframe to a new CSV file
merged_df.to_csv("data/ReactionSmilesECTest.csv", index=False)

In [ ]:
input= "data/SmilesEC.csv"
output='data/ReactionSmiles.csv'

df1 = pd.read_csv(input, sep=',')

#Volem eliminar tota la informació de les reaccions que tinguin algun valor nul en els seus subtrats i productes a la columna smiles i a la columna de ECNumber.

#Primer afegim 0 on tronem valors NaN a la columna Smiles del dataframe.
df1['Smiles'] = df1['Smiles'].fillna(0)
df1["ECNumber"]= df1["ECNumber"].fillna(0)


# generem una llista buida on guardarem el SabioReactionID dels substrats i productes que s'hagin d'elimianr 
list1 = [] 

# iterem el df1   
for i in range(len(df1)): 

    # Si Smiles es igual a 0 afegim el SabioReactionID a la list1 
    if df1.loc[i,'Smiles'] == 0:
        list1.append(df1.loc[i,'SabioReactionID']) 
#Eliminem del df1 tots els substrats i productes que tinguin el SabioReactionID igual que el que s'ha guardat a la llista
df = df1.drop(df1[df1['SabioReactionID'].isin(list1)].index) 

print(df1)

#A partir del dataframe sense missing values generem el dataframe curt per fer proves.
#Generem un dataframe buit amb les columnes SabioID i Reaction de 11 files.
df = df.reset_index()  #la funció rest_index() restableix l'índex del dataframe, estableix una llista del 0 fins a la longitud del dataframe
numrows=16513

df2 = pd.DataFrame(columns=["SabioReactionID", "Reaction"]) #, index=range(numrows))

#Creem dues llistes buides amb tants strings com files (en aquest cas 11 numrows=11). Generem una llista R per guardar els reactius i una llista P per guardar els productes.
R=[""]*(numrows+1)
P=[""]*(numrows+1)
isFirstR=[True]*(numrows+1)
isFirstP=[True]*(numrows+1)

def check_smiles(smiles):
    print(smiles)
    if not smiles:
        print('Wrong SMILES for:',smiles)
        return 0
    return 1

#Fem una iteració per cada fila del dataframe amb el command: for index, row in df.iterrows():
for index, row in df.iterrows():
    rid=row['SabioReactionID']
    name=row["Name"]
    smiles=row["Smiles"]
    if check_smiles(smiles) == 0:
        break
   
    role=row["Role"]
    if role == "Substrate": 
        if isFirstR[rid]:
            R[rid]=R[rid] + smiles 
            isFirstR[rid]= False
        else:
            R[rid]=R[rid] + "." + smiles       
        
    elif role == "Product":
        if isFirstP[rid]:
            P[rid]=P[rid]+smiles
            isFirstP[rid]=False
        else:
            P[rid]=P[rid] +"."+ smiles
    else:
        print("error, name")
    print(R[rid],P[rid])
    df2.at[rid, "SabioReactionID"]=rid

for i in range(numrows):
    print(R[i]+">"+P[i])
    df2.at[i, "Reaction"]=R[i]+">>"+P[i]

  

df2.drop(df2.tail(2).index,inplace=True)
print(df2)

#obtenim ReactionSmiles.csv.
df2.to_csv(output)

In [ ]:
input1="data/ReactionSmiles.csv"
input2='data/ECnumber.csv'

df1 = pd.read_csv(input1, sep=',')
df2 = pd.read_csv(input2, sep="\t")

# Merge the two dataframes on the common column "numbers"
merged_df = pd.merge(df1, df2, on="SabioReactionID")

# Write the merged dataframe to a new CSV file
merged_df.to_csv("data/ReactionSmilesEC.csv", index=False)

In [ ]:
input= smilesfile
output='data/moleculesSmilesTest.csv'

df1 = pd.read_csv(input, sep='\t')

df1['Smiles'] = df1['Smiles'].fillna(0)
# creem uns llista buida.  
list1 = [] 
#smiles=row["Smiles"]   
for i in range(len(df1)): 

    # comprovem si Smiles és igual a 0, llavors afegim el valors SabioReactionID a la llista list1 
    if df1.loc[i,'Smiles'] == 0: 
        list1.append(df1.loc[i,'SabioReactionID']) 

df = df1.drop(df1[df1['SabioReactionID'].isin(list1)].index) 
# Print the updated dataframe 
#print(df)

dfmolecules = df.drop_duplicates(subset="Smiles")

print(dfmolecules)
dfmolecules.to_csv(output)

In [ ]:
from rdkit import Chem
from rdkit.Chem import Crippen, Descriptors
import matplotlib.pyplot as plt

df1 = pd.read_csv('data/moleculesSmilesTest.csv', sep=',')
df1['mol']=df1['Smiles'].apply(lambda x: Chem.MolFromSmiles(x)) #transformem la columna "Smiles" en mol per començar a treballar amb RDKit
print(type(df1['mol'][0])) #comprovem que hem genereat la classe rdkit.Chem.rdchem.Mol

from rdkit import Chem
from rdkit.Chem import Crippen, Descriptors
output='data/fingerprintTest.csv'

mols = df1['mol']

def smiles_to_mol(smiles):
    mol = Chem.MolFromSmiles(smiles)
    return mol


def features(mol,data):
    #Extract properties using RDKit functions
    smiles = Chem.MolToSmiles(mol)
    formula = Chem.rdMolDescriptors.CalcMolFormula(mol)
    mw= Chem.rdMolDescriptors.CalcExactMolWt(mol) #pes molecular
    logp=Chem.Descriptors.MolLogP(mol)
    tpsa = Chem.rdMolDescriptors.CalcTPSA(mol)
    num_rc=Chem.Descriptors.RingCount(mol)
    num_AR=Chem.Descriptors.NumAromaticRings(mol)
    num_HAC=Chem.Descriptors.HeavyAtomCount(mol)
    num_AR=Chem.Descriptors.NumAliphaticRings(mol)
    CSP3=Chem.Descriptors.FractionCSP3(mol)
    BJI=Chem.Descriptors.BalabanJ(mol)
    num_RB=Chem.Descriptors.NumRotatableBonds(mol)
    num_hbd = Chem.rdMolDescriptors.CalcNumHBD(mol)
    num_hba = Chem.rdMolDescriptors.CalcNumHBA(mol)
   

    #Append results in a data list
    data.append([mw,logp, tpsa, num_rc, num_AR, num_HAC, num_AR, CSP3, BJI, num_RB, num_hbd, num_hba])
    return

data = []

for mol in mols:
    features(mol,data)
llistadescriptors=['MW','logP',"TPSA", "Rings Count", "Aromatic Rings", "Heavy atoms cout", "Alifatic Rings",  "CSP3",  "BalabanJ", "RotatableBonds", "H-bond donor", "H-bond acceptor"]
#We have a list with information. Creation of a Dataframe:
df= pd.DataFrame(data,columns=llistadescriptors)
#df["SabioReactionID"]= df1["SabioReactionID"]
#df["Role"]=df1["Role"]
#df["Name"]=df1["Name"]
df["Smiles"]=df1["Smiles"]

print(df)

df.to_csv(output)

In [ ]:
input1="data/reactions_with_descriptors.csv"
input2='data/ECNumber.csv'

df1 = pd.read_csv(input1, sep=',')
df2 = pd.read_csv(input2, sep="\t")

# Merge the two dataframes on the common column "numbers"
merged_df = pd.merge(df1, df2, on="SabioReactionID")

# Write the merged dataframe to a new CSV file
merged_df.to_csv("data/R_w_D_T.csv", index=False)

In [ ]:
hist1=df.hist(column='MW')
hist2=df.hist(column='logP')
hist3=df.hist(column='TPSA')
hist4=df.hist(column='Rings Count')
hist5=df.hist(column='Aromatic Rings')
hist6=df.hist(column='CSP3')
hist7=df.hist(column='BalabanJ')
hist8=df.hist(column='RotatableBonds')
hist9=df.hist(column='H-bond donor')
hist10=df.hist(column='H-bond acceptor')

In [ ]:
import pandas as pd

input2 = 'data/FingerprintTest.csv'
input1 = 'data/ReactionSmilesTest.csv'
output= 'data/output_file.csv'

df2 = pd.read_csv(input2, sep=',')
df1 = pd.read_csv(input1, sep=',')

#generem un diccionari que assigni les cadenes SMILES als valors dels descriptors
molecule_descriptors = df2.set_index('Smiles').to_dict('index')

# Funció per afegir descriptors a les reaccions.
def add_descriptors(row):
    reaction_smiles = row['Reaction']
    reactants, products = reaction_smiles.split('>>')
    reactant_smiles = reactants.split('.')
    product_smiles = products.split('.')
    llistadescriptors=['MW','logP',"TPSA", "Rings Count", "Aromatic Rings", "Heavy atoms cout", "Alifatic Rings",  "CSP3",  "BalabanJ", "RotatableBonds", "H-bond donor", "H-bond acceptor"]
    reactrow=pd.Series([0]*12,index=llistadescriptors)
    productrow=pd.Series([0]*12,index=llistadescriptors)
    
    
    # Recorre cada reactiu i producte i s'afegeixen els descriptors si aquests es troben al diccionari de descriptors de molècules
    for reactant in reactant_smiles:
       # print(reactant)
        ser2=pd.Series(molecule_descriptors[reactant]).drop(labels=['Unnamed: 0']) #eliminem la columna Unnamed i fem la sumna  dels valors de cada variable dels reactius.
        #print('pd.Series\n',ser2)
        reactrow=reactrow.add(ser2)
    for product in product_smiles:
        ser2=pd.Series(molecule_descriptors[product]).drop(labels=['Unnamed: 0']) #eliminem la columna Unnamed i fem la suma dels valors de cada variable dels productes.
        productrow=productrow.add(ser2)
    row = pd.concat([row, reactrow, productrow])#, ignore_index=True)
    return row



# Apliquem la funció add_descriptors a cada fila del dataframe de reaccions.
reactions_df = df1.apply(add_descriptors, axis=1)

#print(reactions_df)

# escrivim el dataframe en un csv.
reactions_df.to_csv('data/reactions_with_descriptors.csv', index=False)

In [ ]:
import pandas as pd

input2 = 'data/FingerprintTest.csv'
input1 = 'data/ReactionSmilesTest.csv'
output= 'data/output_file.csv'

df2 = pd.read_csv(input2, sep=',')
df1 = pd.read_csv(input1, sep=',')

#create a dictionary mapping SMILES strings to descriptor values
molecule_descriptors = df2.set_index('Smiles').to_dict('index')

# function to add descriptors to reactions
def add_descriptors(row):
    reaction_smiles = row['Reaction']
    reactants, products = reaction_smiles.split(">")
    
    reactant_smiles = reactants.split('.')
    product_smiles = products.split('.')
    
    # loop over each reactant and product and add descriptors if they exist in the molecule_descriptors dictionary
    for reactant in reactant_smiles:
        if reactant in molecule_descriptors:
            row = row.append(pd.Series(molecule_descriptors[reactant]), ignore_index=True)
    for product in product_smiles:
        if product in molecule_descriptors:
            row = row.append(pd.Series(molecule_descriptors[product]), ignore_index=True)
    
    return row

# apply the add_descriptors function to each row in the reactions dataframe
reactions_df = df1.apply(add_descriptors, axis=1)

# write the updated reactions dataframe to a new CSV file
reactions_df.to_csv('reactions_with_descriptors.csv', index=False)